<a href="https://colab.research.google.com/github/jinsusong/21-study-tensorflow/blob/main/07%20%ED%85%8D%EC%8A%A4%ED%8A%B8%20%EB%B6%84%EC%84%9D%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 텐서플로2와 머신러닝으로 시작하는 자연어 처리 
### 로지스틱 회귀부터 BERT와 GPT2 까지


### 텍스트 분류 



: 특정 텍스트를 사람들이 정한 몇 가지 범주 중 어느 범주에 속하는지 분류하는 문제 
- 이진 분류 
- 다중 범주 분류




지도 학습을 통한 텍스트 분류 
- 나이브 베이즈 분류 
- 서포트 벡터 머신
- 신경망
- 선형 분류 
- 로지스틱 분류
- 랜덤 포레스트


비지도 학습을 통한 텍스트 분류 
- k-평균 군집화(k-means clustering)
- 계측적 군집화

### 텍스트 유사도 

유사도 측정을 위해 정량화하는 방법
- 단순히 같은 단어의 개수를 사용해서 유사도를 판단하는 방법
- 형태소로 나누어 형태소를 비교하는 방법
- 자소 단위로 나누어 단어를 비교하는 방법 

대표적으로 사용되는 4개의 유사도 측정 방법
- 자카드 유사도
- 유클리디언 유사도
- 맨해튼 유사도
- 코사인 유사도 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
sent = ("휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다.","폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니다.")
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sent) # 문장 벡터화 진행

idf = tfidf_vectorizer.idf_
print(dict(zip(tfidf_vectorizer.get_feature_names(),idf)))
# 각 수치에 대한 값 시각화 


자카드 유사도 : 두 문장을 각각 단어의 집합으로 만든 뒤 두 집합을 통해 유사도를 측정하는 방식 중 하나

 - 유사도를 측정하는 방식은 두 집합의 교집합인 공통된 단어의 개수를 두 집합의 합집합, 즉 전체 단어의 수로 나누면 됨 
 

코사인 유사도 : 두 개의 벡터값에서 코사인 각도를 구하는 방법
- 단순히 좌표상의 거리를 구하는 다른 유사도 측정 방법에 비해 코사인 유사도는 두 벡터 간의 각도를 구하는 것이라서 방향성의 개념이 더해지기 때문에 일반적으로 성능이 더 좋다 .
- 두 문장이 유사하다면 같은 방향으로 가르키고 , 유사하지 않을수록 직교로 표현됨


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2]) 


유클리디언 유사도 : 가장 기본적인 거리를 측정하는 유사도 공식 
- n차원 공간에서 두 점 사이의 거리를 구하는 방식 


In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(tfidf_matrix[0:1], tfidf_matrix[1:2])

맨해튼 유사도 
: 맨새튼 거리를 통해 유사도를 측정하는 방법
- 사각형 격자로 이뤄진 지도에서 출발점에서 도착점까지를 가로지르지 않고 갈 수 있는 최단거리를 구하는 공식 

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances

manhattan_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

자연어 생성  : 번역, 챗봇 등의 분야에서 사용중 

데이터 이해하기 

In [ ]:
import os
import re 

import pandas as pd
import tensorflow as tf 
from tensorflow.keras import utils

data_set = tf.keras.utils.get_file(
    fname="imdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True)


In [ ]:
def directory_data(directory):
    data = {}
    data["review"]= []
    for file_path in os.listdir(directory):
        with open(os.path.join(directory, file_path),"r")as file:
            data["review"].append(file.read())

    return pd.DataFrame.from_dict(data)

In [ ]:
def data(directory):
    pos_df = directory_data(os.path.join(directory, "pos"))
    neg_df = directory_data(os.path.join(directory, "neg"))
    pos_df["sentiment"]=1
    neg_df["sentiment"]=0

    return pd.concat([pos_df, neg_df])
    

In [ ]:
train_df = data(os.path.join(os.path.dirname(data_set),"aclImdb","train"))
test_df = data(os.path.join(os.path.dirname(data_set),"aclImdb", "test"))

In [ ]:
train_df.head()

In [ ]:
reviews = list(train_df['review'])


In [ ]:
#문자열 문장 리스트를 토크나이징 
tokenized_reviews = [r.split() for r in reviews]

#토크나이징된 리스트에 대한 각 길이를 저장 
review_len_by_token = [len(t) for t in tokenized_reviews]

#토크나이징된 것을 붙여서 음절의 길이를 저장 
review_len_by_eumjeol = [len(s.replace(' ',''))for s in reviews]



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.hist(review_len_by_token, bins=50, alpha=0.5, color='r', label='word')
plt.hist(review_len_by_eumjeol, bins=50, alpha=0.5, color='b', label='alphabet')
plt.yscale('log',nonposy='clip')
plt.title('Review Length Histogram')
plt.xlabel('Review Length')
plt.ylabel('Number of Reviews')

In [ ]:
import numpy as np 
print('문장 최대 길이: {}'.format(np.max(review_len_by_token)))
print('문장 최소 길이:{}'.format(np.min(review_len_by_token)))
print('문장 평균 길이 : {}'.format(np.mean(review_len_by_token)))
print('문장 길이 표준편차 : {:.2f}'.format(np.std(review_len_by_token)))
print('문장 중간 길이: {}'.format(np.median(review_len_by_token)))
#사분위의 대한 경우는 0~100 스케일로 돼 있음 
print('제 1사분위 길이 : {} '.format(np.percentile(review_len_by_token,25)))
print('제 3사분위 길이 : {} '.format(np.percentile(review_len_by_token,75)))

In [ ]:
plt.figure(figsize=(12,5))
plt.boxplot([review_len_by_token],
            labels=['token'],
            showmeans=True)

In [ ]:
plt.figure(figsize=(12,5))
plt.boxplot([review_len_by_eumjeol],
            labels=['Eumjeol'],
            showmeans=True)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

wordcloud= WordCloud(stopwords = STOPWORDS, background_color = 'black', width= 800, height= 600).generate(' '.join(train_df['review']))

plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sentiment = train_df['sentiment'].value_counts()
fig, axe = plt.subplots(ncols=1)
fig.set_size_inches(6,3)
sns.countplot(train_df['sentiment'])